In [ ]:
### save supplementary data
import os, sys
currentdir = os.path.dirname(os.path.realpath("__file__"))
parentdir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath("__file__"))))
sys.path.append(parentdir)

from joblib import Parallel, delayed
import numpy as np
import pickle 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchmetrics
import seaborn as sns
import glob
from tqdm import tqdm
from copy import deepcopy
from src.transformer.metrics import AUL, CorrectedMCC, CorrectedBAcc, CorrectedF1
from src.analysis.metrics.cartesian import cartesian_jit

from torchmetrics import MatthewsCorrCoef, F1Score, Accuracy
from sklearn.utils import resample
from scipy.stats import median_abs_deviation as mad
import time
import cmcrameri.cm as cmc
import matplotlib.colors as clr
from matplotlib.ticker import MultipleLocator, LinearLocator, AutoMinorLocator
plt.rcParams["grid.linestyle"] =  ":"
plt.rcParams["axes.edgecolor"] = "gray"
plt.rcParams["axes.linewidth"] = 0.7

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Arial"

In [ ]:
def load_stats(path):
    data = {}
    with open(path + "prb.npy", "rb") as f:
        data["prb"] = np.load(f)
    with open(path + "trg.npy", "rb") as f:
        data["trg"] = np.load(f)
    with open(path + "id.npy", "rb") as f:
        data["id"] = np.load(f)
    return data

In [ ]:
v = "4.02"
save_path = r"../analysis/plots/%s/" %v

In [ ]:
np.random.seed(0)
data = { "ltv": load_stats(r"..\\predictions\\v15\\cls\\eos_l2v\\%s\\"%v),
               "rnn": load_stats(r"..\\predictions\\v15\\cls\\eos_rnn\\1.0\\"),
               "nn": load_stats(r"..\\predictions\\v15\\tcls\\eos_tab\\1.0\\"),
               "logistic": load_stats(r"..\\predictions\\v15\\tcls\\eos_tab\\3.1\\"),
               "table": load_stats(r"..\\predictions\\v15\\tcls\\eos_tab\\3.2\\")}
np.random.seed(0)
data["rnd"]=np.random.uniform(size=data["rnn"]["trg"].shape[0])


In [ ]:
## Data for the overall comparison of performances
np.random.seed(0)
df = pd.DataFrame({"true_label (deceased)": data["ltv"]["trg"].astype(bool),
              "l2v_pred":data["ltv"]["prb"] >= 0.5,
              "rnn_pred":data["rnn"]["prb"] >=0.5,
              "nn_pred": data["nn"]["prb"] >=0.5,
              "mt_pred": data["table"]["prb"] >=0.5,
              "lr_pred": data["logistic"]["prb"] >=0.5,
              "rnd_pred": data["rnd"] >= 0.5,
              "mjr_pred": np.full(shape = data["rnd"].shape, fill_value=True, dtype=bool)}).sample(frac=1).reset_index(drop=True)
df.to_csv(save_path + "survivalA_data_overall_comparison.csv")
df.head()